In [12]:
# Importing python Library
import mne, os, time, pickle, warnings, itertools, copy, sys, shutil
from mne.filter import filter_data as bandpass_filter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import svm
from scipy.signal import welch, butter, lfilter, sosfilt, sosfreqz, freqz
from scipy.integrate import simps
from scipy.stats import f_oneway
from tqdm import tqdm
from mne.preprocessing import ICA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from skfeature.utility.construct_W import construct_W
from scipy.sparse import diags
warnings.filterwarnings('ignore')

In [21]:
def SignalPreProcess(eeg_rawdata):
    """
    :param eeg_rawdata: numpy array with the shape of (n_channels, n_samples)
    :return: filtered EEG raw data
    """
    assert eeg_rawdata.shape[0] == 32
    eeg_rawdata = np.array(eeg_rawdata)

    ch_names = ["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", 
                "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8",
                "PO4", "O2"]
  
    info = mne.create_info(ch_names = ch_names, ch_types = ['eeg' for _ in range(32)], sfreq = 128, verbose=False)
    raw_data = mne.io.RawArray(eeg_rawdata, info, verbose = False)
    raw_data.load_data(verbose = False).filter(l_freq = 4, h_freq = 48, method = 'iir', verbose = False)
    ica = ICA(n_components = N_C, random_state = 97, verbose = False)
    ica.fit(raw_data)
    eog_indices, eog_scores = ica.find_bads_eog(raw_data, ch_name = 'Fp1', verbose = None)
    #ica.plot_scores(eog_scores)
    a = abs(eog_scores).tolist()
    if(droping_components == 'one'):
        ica.exclude = [a.index(max(a))]
    else: # find two maximum scores
        a_2 = a.copy()
        a.sort(reverse = True)
        exclude_index = []
        for i in range(0, 2):
            for j in range(0, len(a_2)):
                if(a[i]==a_2[j]):
                    exclude_index.append(j)
        ica.exclude = exclude_index
    ica.apply(raw_data, verbose = False)
    # common average reference
    raw_data.set_eeg_reference('average', ch_type = 'eeg')#, projection = True)
    filted_eeg_rawdata = np.array(raw_data.get_data())
    return filted_eeg_rawdata

def signal_pro(input_data):
    for i in range(input_data.shape[0]):
        input_data[i] = SignalPreProcess(input_data[i].copy())
    return input_data

In [22]:
sf = 128 # sampling frequency 128 Hz

In [23]:
theta_band_range = (4, 8)   # drownsiness, emotional connection, intuition, creativity
alpha_band_range = (8, 12)  # reflection, relaxation
beta_band_range = (12, 30)  # concentration, problem solving, memory
gamma_band_range = (30, 45) # cognition, perception, learning, multi-tasking

In [24]:
def get_csv_file(subject, reduced_data, labels):
    eeg_theta, eeg_alpha, eeg_beta, eeg_gamma = [], [], [], []
    channels_theta, channels_alpha, channels_beta, channels_gamma = [], [], [], []
    for video_no in range (len(reduced_data)):
        for channel_no in range (len(reduced_data[0])):
            input_data = reduced_data[video_no, channel_no].copy()
            # Compute the modified periodogram (Welch)
            freqs, psd = welch(input_data, sf)
            # Find closest indices of band in frequency vector
            theta_idx_band = np.logical_and(freqs >= theta_band_range[0], freqs <= theta_band_range[1])
            alpha_idx_band = np.logical_and(freqs >= alpha_band_range[0], freqs <= alpha_band_range[1])
            beta_idx_band = np.logical_and(freqs >= beta_band_range[0], freqs <= beta_band_range[1])
            gamma_idx_band = np.logical_and(freqs >= gamma_band_range[0], freqs <= gamma_band_range[1])

            eeg_theta.append(np.mean(psd[theta_idx_band])) #mean of the frequency bands)
            eeg_alpha.append(np.mean(psd[alpha_idx_band]))
            eeg_beta.append(np.mean(psd[beta_idx_band]))
            eeg_gamma.append(np.mean(psd[gamma_idx_band]))
    eeg_theta = np.reshape(eeg_theta, (40, 32)) # 40 videos and 32 channels theta band power
    eeg_alpha = np.reshape(eeg_alpha, (40, 32))
    eeg_beta = np.reshape(eeg_beta, (40, 32))
    eeg_gamma = np.reshape(eeg_gamma, (40, 32))
    
    for i in range(0, len(eeg_channels)):
        channels_theta.append(eeg_channels[i] + '_theta')
        channels_alpha.append(eeg_channels[i] + '_alpha')
        channels_gamma.append(eeg_channels[i] + '_gamma')
        channels_beta.append(eeg_channels[i] + '_beta')
        
    df_theta = pd.DataFrame(eeg_theta, columns = channels_theta)
    df_alpha = pd.DataFrame(eeg_alpha, columns = channels_alpha)
    df_beta = pd.DataFrame(eeg_beta, columns = channels_beta)
    df_gamma = pd.DataFrame(eeg_gamma, columns = channels_gamma)
    
    # make a directory to save the csv file
    new_path = newpath + subject
    try:
        os.mkdir(new_path)
        os.mkdir(new_path + '/rawfiles')
    except:
        # If directory exists then delete that directory
        shutil.rmtree(new_path)
        # then make the new directory
        os.mkdir(new_path)
        os.mkdir(new_path + '/rawfiles')
    df_theta.to_csv(new_path + '/rawfiles/' + subject + '_theta.csv', index = False, encoding = 'utf-8-sig')
    df_alpha.to_csv(new_path + '/rawfiles/' + subject + '_alpha.csv', index = False, encoding = 'utf-8-sig')
    df_beta.to_csv(new_path + '/rawfiles/' + subject  + '_beta.csv', index = False, encoding = 'utf-8-sig')
    df_gamma.to_csv(new_path + '/rawfiles/' + subject + '_gamma.csv', index = False, encoding = 'utf-8-sig')
    #===========================   ALL Bands   ==================================
    frames = [df_theta, df_alpha, df_beta, df_gamma]
    all_bands = pd.concat(frames, axis = 1) # join these 4 data frame columns wise, row is fixed
    all_bands.to_csv(new_path + '/rawfiles/' + subject + '.csv', index = False, encoding = 'utf-8-sig')
    all_bands_valence, all_bands_arousal, all_bands_all = all_bands.copy(), all_bands.copy(), all_bands.copy()
    all_bands_valence['valence'] = emotion_label(labels, 'valence')
    all_bands_arousal['arousal'] = emotion_label(labels, 'arousal')
    all_bands_all['all'] = emotion_label(labels, 'all')
    all_bands_valence.to_csv(new_path + '/' + subject + '_valence.csv', index = False, encoding = 'utf-8-sig')
    all_bands_arousal.to_csv(new_path + '/' + subject + '_arousal.csv', index = False, encoding = 'utf-8-sig')
    all_bands_all.to_csv(new_path + '/' + subject + '_all.csv', index = False, encoding = 'utf-8-sig')

In [25]:
def emotion_label(labels, class_label):
    """
    This function gives the valence/arousal and HVHA/HVLA/LAHV/LALV class labels
    """
    em_labels = []
    if(class_label == 'valence'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                em_labels.append(1)
            else: # low valence
                em_labels.append(0)
        return em_labels
    elif(class_label == 'arousal'):
        for i in range(0, labels.shape[0]):
            if (labels[i][1]>5): # high arousal
                em_labels.append(1)
            else: # low arousal
                em_labels.append(0)
        return em_labels
    elif(class_label == 'all'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(1) # HVHA
                else:
                    em_labels.append(0) # HVLA
            else: # low valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(2) # LVHA
                else: # low arousal
                    em_labels.append(3) # LVLA
        return em_labels

In [26]:
def fisher_score(X, y):
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1
    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels
    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature
    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """
    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)

def get_fisher_score(subject_name, em_labels):
    data_theta = pd.read_csv(band_wise_data_path + subject_name + '_theta.csv')
    data_alpha = pd.read_csv(band_wise_data_path + subject_name + '_alpha.csv')
    data_beta = pd.read_csv(band_wise_data_path + subject_name + '_beta.csv')
    data_gamma = pd.read_csv(band_wise_data_path + subject_name + '_gamma.csv')
    
    X_theta, X_alpha = np.array(data_theta), np.array(data_alpha)
    X_beta, X_gamma = np.array(data_beta), np.array(data_gamma)
    y = np.array(em_labels)
    # apply scalling of the given data
    scaler = MinMaxScaler()
    scaler.fit(X_theta)
    X_theta = scaler.transform(X_theta)
    scaler.fit(X_alpha)
    X_alpha = scaler.transform(X_alpha)
    scaler.fit(X_beta)
    X_beta = scaler.transform(X_beta)
    scaler.fit(X_gamma)
    X_gamma = scaler.transform(X_gamma)
    
    fscore_theta, fscore_alpha = fisher_score(X_theta, y), fisher_score(X_alpha, y)
    fscore_beta, fscore_gamma = fisher_score(X_beta, y), fisher_score(X_gamma, y)
    
    # Total Avearge F-Score (Theta, Alpha, Beta, Gamma)
    final_f_score = (fscore_theta + fscore_alpha + fscore_beta + fscore_gamma)/4
    fvalues = pd.Series(final_f_score)
    fvalues.index = eeg_channels
    fvalues.sort_values(ascending = False)
    fvalues.to_csv('fscore_final.csv')
    # for visualization run the below code
    # fvalues.sort_values(ascending = False).plot.bar(figsize=(10,8))
    df = fvalues.sort_values(ascending = False)
    da = pd.DataFrame(df)
    da.to_csv('channel_rank.csv')
    cr = pd.read_csv('channel_rank.csv')
    sort_channel_name = list(cr['Unnamed: 0'])
    os.remove('channel_rank.csv') # delete the csv file
    return sort_channel_name

def emotion_label(labels, class_label):
    em_labels = []
    if(class_label == 'valence'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                em_labels.append(1)
            else: # low valence
                em_labels.append(0)
        return em_labels
    elif(class_label == 'arousal'):
        for i in range(0, labels.shape[0]):
            if (labels[i][1]>5): # high arousal
                em_labels.append(1)
            else: # low arousal
                em_labels.append(0)
        return em_labels
    elif(class_label == 'all'):
        for i in range(0, labels.shape[0]):
            if (labels[i][0]>5): # high valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(1) # HVHA
                else:
                    em_labels.append(0) # HVLA
            else: # low valence
                if(labels[i][1]>5): # high arousal
                    em_labels.append(2) # LVHA
                else: # low arousal
                    em_labels.append(3) # LVLA
        return em_labels
    
#Loading the dataset
def svmclassifier(channel_name, data):
    channel_names = []
    for i in range(0, len(channel_name)):
        draft = channel_name[i]
        channel_names.append(draft + "_alpha")
        channel_names.append(draft + "_beta")
        channel_names.append(draft + "_gamma")
        channel_names.append(draft + "_theta")
    x, y = data[channel_names], np.array(em_labels)
    # Implementing cross validation
    k = 40
    kf = KFold(n_splits = k, shuffle = False)
    acc_score = []
    for train_index , test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index,:],x.iloc[test_index,:]
        y_train, y_test = y[train_index] , y[test_index]
        model = svm.SVC(kernel = 'poly')
        model.fit(x_train, y_train)
        pred_values = model.predict(x_test)
        #pred_values = model.predict(x_test)
        acc = accuracy_score(pred_values , y_test)
        acc_score.append(acc)
    avg_acc_score = sum(acc_score)/k
    return avg_acc_score

def growing_phase(channel_name):
    cn = channel_name[0]
    acc = svmclassifier([cn], data.copy())
    cn_list = []
    cn_list.append(cn)
    sort_cn = []
    for i in range(1, len(channel_name)):
        cur_cn = channel_name[i]
        cn_list.append(cur_cn)
        cur_acc = svmclassifier(cn_list, data.copy())
        if(cur_acc<acc):
            cn_list.remove(cur_cn)
        else:
            acc = cur_acc
    print('Accuracy in Growing Phase: ', acc)
    print('No of selected channels in Growing Phase: ', len(cn_list))
    print('Channels selected in Growing Phase: ', cn_list)
    return cn_list

In [27]:
subject_names = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 
                 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21',
                 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32']
eeg_channels = np.array(['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 
                         'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 
                         'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2'])
class_labels = ['valence', 'arousal', 'all']
# deap dataset path # put the path in which deap dataset files are present
deap_dataset_path = '/Users/shyammarjit/Desktop/Brain Computer Interface/Deap Dataset/'
# put the path location of datfiles folder s.t. subject wise folder should contain datafiles
datafiles_path = '/Users/shyammarjit/Desktop/Brain Computer Interface/Hybrid Sequential Forward channel selection (HSFCS)/Subject Dependent/band_45_fir_None_one_new/'
# put the path in which you want to save the csv file
save_csv_path = '/Users/shyammarjit/Desktop/Brain Computer Interface/Hybrid Sequential Forward channel selection (HSFCS)/Subject Dependent'

In [11]:
# fir filter
N_C = None
droping_components = 'one'

for subject in subject_names[0:32]:
    newpath = save_csv_path + '/band_45_fir_None_one_new/'
    try:
        # If the directory already exists then don't make any new dirctory
        os.mkdir(newpath)
    except:
        pass
    # load the dataset
    with open(deap_dataset_path + subject + '.dat', 'rb') as f:
        raw_data = pickle.load(f, encoding = 'latin1')
    # raw_data has two key 'data' and 'labels'
    data = raw_data['data']
    labels = raw_data['labels']
    # we are excluding 3s pre base line i.e. first 3*128 = 384 data points from time series data
    reduced_eeg_data  = data[0:40, 0:32, 384:8064]
    reduced_data = signal_pro(reduced_eeg_data.copy())
    get_csv_file(subject, reduced_data, labels)
    print('Done: ', subject)
    
    
    #break

    subject_name = subject
    print("\n")
    print("="*90)
    print('Subject Name: ', subject_name)
    print("="*90)
    band_wise_data_path =  datafiles_path + subject_name + '/rawfiles/'
    mypath = band_wise_data_path + subject_name
    data = pd.read_csv(mypath + '.csv')
    data_col = data.columns
    # Do min max scalling
    scaler = MinMaxScaler()
    scaler.fit(np.array(data))
    data_arr = scaler.transform(np.array(data))
    data = pd.DataFrame(data_arr, columns = data_col)
    for class_label in class_labels:
        print('Class Label: ', class_label)
        link = deap_dataset_path + subject_name + '.dat'
        with open(link, 'rb') as f:
            raw_data = pickle.load(f, encoding = 'latin1')
        labels = raw_data['labels']
        em_labels = emotion_label(labels, class_label) # get the emotion labels
        channel_name = get_fisher_score(subject_name, em_labels) # get the fscore of all channels
        optimal_channel_name = growing_phase(channel_name) # get the optimal channels name

Done:  s01


Subject Name:  s01
Class Label:  valence
Accuracy in Growing Phase:  0.725
No of selected channels in Growing Phase:  8
Channels selected in Growing Phase:  ['O1', 'PO3', 'P7', 'Oz', 'CP2', 'CP1', 'AF4', 'AF3']
Class Label:  arousal
Accuracy in Growing Phase:  0.775
No of selected channels in Growing Phase:  2
Channels selected in Growing Phase:  ['P7', 'T7']
Class Label:  all
Accuracy in Growing Phase:  0.55
No of selected channels in Growing Phase:  3
Channels selected in Growing Phase:  ['P7', 'PO3', 'AF4']
Done:  s02


Subject Name:  s02
Class Label:  valence
Accuracy in Growing Phase:  0.7
No of selected channels in Growing Phase:  8
Channels selected in Growing Phase:  ['Fp2', 'FC1', 'Oz', 'CP6', 'Cz', 'T8', 'P8', 'CP2']
Class Label:  arousal
Accuracy in Growing Phase:  0.725
No of selected channels in Growing Phase:  8
Channels selected in Growing Phase:  ['P7', 'F3', 'Fp2', 'T8', 'FC1', 'O2', 'Cz', 'CP6']
Class Label:  all
Accuracy in Growing Phase:  0.575
No of se

Done:  s12


Subject Name:  s12
Class Label:  valence
Accuracy in Growing Phase:  0.75
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['CP1', 'CP2', 'P7', 'P4', 'Pz', 'T8']
Class Label:  arousal
Accuracy in Growing Phase:  0.875
No of selected channels in Growing Phase:  13
Channels selected in Growing Phase:  ['AF4', 'FC6', 'Cz', 'T8', 'Oz', 'CP6', 'PO4', 'CP1', 'CP2', 'AF3', 'FC1', 'C4', 'PO3']
Class Label:  all
Accuracy in Growing Phase:  0.625
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['PO4', 'FC2', 'CP5', 'F4', 'PO3', 'T8']
Done:  s13


Subject Name:  s13
Class Label:  valence
Accuracy in Growing Phase:  0.75
No of selected channels in Growing Phase:  12
Channels selected in Growing Phase:  ['FC2', 'P7', 'Fz', 'O2', 'P4', 'Cz', 'P8', 'FC1', 'Oz', 'AF3', 'Pz', 'F7']
Class Label:  arousal
Accuracy in Growing Phase:  0.85
No of selected channels in Growing Phase:  4
Channels selected in Growing Phase:  ['T8', 

KeyboardInterrupt: 

In [20]:
# iir 
N_C = None
droping_components = 'one'

for subject in subject_names[0:32]:
    newpath = save_csv_path + '/band_45_fir_None_one_new/'
    try:
        # If the directory already exists then don't make any new dirctory
        os.mkdir(newpath)
    except:
        pass
    # load the dataset
    with open(deap_dataset_path + subject + '.dat', 'rb') as f:
        raw_data = pickle.load(f, encoding = 'latin1')
    # raw_data has two key 'data' and 'labels'
    data = raw_data['data']
    labels = raw_data['labels']
    # we are excluding 3s pre base line i.e. first 3*128 = 384 data points from time series data
    reduced_eeg_data  = data[0:40, 0:32, 384:8064]
    reduced_data = signal_pro(reduced_eeg_data.copy())
    get_csv_file(subject, reduced_data, labels)
    print('Done: ', subject)
    
    
    #break

    subject_name = subject
    print("\n")
    print("="*90)
    print('Subject Name: ', subject_name)
    print("="*90)
    band_wise_data_path =  datafiles_path + subject_name + '/rawfiles/'
    mypath = band_wise_data_path + subject_name
    data = pd.read_csv(mypath + '.csv')
    data_col = data.columns
    # Do min max scalling
    scaler = MinMaxScaler()
    scaler.fit(np.array(data))
    data_arr = scaler.transform(np.array(data))
    data = pd.DataFrame(data_arr, columns = data_col)
    for class_label in class_labels:
        print('Class Label: ', class_label)
        link = deap_dataset_path + subject_name + '.dat'
        with open(link, 'rb') as f:
            raw_data = pickle.load(f, encoding = 'latin1')
        labels = raw_data['labels']
        em_labels = emotion_label(labels, class_label) # get the emotion labels
        channel_name = get_fisher_score(subject_name, em_labels) # get the fscore of all channels
        optimal_channel_name = growing_phase(channel_name) # get the optimal channels name

Done:  s01


Subject Name:  s01
Class Label:  valence
Accuracy in Growing Phase:  0.625
No of selected channels in Growing Phase:  2
Channels selected in Growing Phase:  ['Fz', 'FC2']
Class Label:  arousal
Accuracy in Growing Phase:  0.75
No of selected channels in Growing Phase:  9
Channels selected in Growing Phase:  ['PO3', 'O1', 'AF4', 'Fz', 'F8', 'F7', 'T7', 'P4', 'O2']
Class Label:  all
Accuracy in Growing Phase:  0.525
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['P3', 'PO3', 'P7', 'CP5', 'AF4', 'Fp1']
Done:  s02


Subject Name:  s02
Class Label:  valence
Accuracy in Growing Phase:  0.725
No of selected channels in Growing Phase:  9
Channels selected in Growing Phase:  ['Fp2', 'FC1', 'Cz', 'Oz', 'T8', 'P8', 'AF4', 'F8', 'FC6']
Class Label:  arousal
Accuracy in Growing Phase:  0.65
No of selected channels in Growing Phase:  3
Channels selected in Growing Phase:  ['P7', 'C3', 'FC6']
Class Label:  all
Accuracy in Growing Phase:  0.525
No of sel

KeyboardInterrupt: 

In [28]:
# 48 iir
# fir filter
N_C = None
droping_components = 'one'

for subject in subject_names[0:32]:
    newpath = save_csv_path + '/band_45_fir_None_one_new/'
    try:
        # If the directory already exists then don't make any new dirctory
        os.mkdir(newpath)
    except:
        pass
    # load the dataset
    with open(deap_dataset_path + subject + '.dat', 'rb') as f:
        raw_data = pickle.load(f, encoding = 'latin1')
    # raw_data has two key 'data' and 'labels'
    data = raw_data['data']
    labels = raw_data['labels']
    # we are excluding 3s pre base line i.e. first 3*128 = 384 data points from time series data
    reduced_eeg_data  = data[0:40, 0:32, 384:8064]
    reduced_data = signal_pro(reduced_eeg_data.copy())
    get_csv_file(subject, reduced_data, labels)
    print('Done: ', subject)
    
    
    #break

    subject_name = subject
    print("\n")
    print("="*90)
    print('Subject Name: ', subject_name)
    print("="*90)
    band_wise_data_path =  datafiles_path + subject_name + '/rawfiles/'
    mypath = band_wise_data_path + subject_name
    data = pd.read_csv(mypath + '.csv')
    data_col = data.columns
    # Do min max scalling
    scaler = MinMaxScaler()
    scaler.fit(np.array(data))
    data_arr = scaler.transform(np.array(data))
    data = pd.DataFrame(data_arr, columns = data_col)
    for class_label in class_labels:
        print('Class Label: ', class_label)
        link = deap_dataset_path + subject_name + '.dat'
        with open(link, 'rb') as f:
            raw_data = pickle.load(f, encoding = 'latin1')
        labels = raw_data['labels']
        em_labels = emotion_label(labels, class_label) # get the emotion labels
        channel_name = get_fisher_score(subject_name, em_labels) # get the fscore of all channels
        optimal_channel_name = growing_phase(channel_name) # get the optimal channels name

Done:  s01


Subject Name:  s01
Class Label:  valence
Accuracy in Growing Phase:  0.775
No of selected channels in Growing Phase:  4
Channels selected in Growing Phase:  ['Fz', 'F4', 'PO3', 'Fp1']
Class Label:  arousal
Accuracy in Growing Phase:  0.875
No of selected channels in Growing Phase:  3
Channels selected in Growing Phase:  ['P7', 'CP1', 'Oz']
Class Label:  all
Accuracy in Growing Phase:  0.5
No of selected channels in Growing Phase:  3
Channels selected in Growing Phase:  ['P7', 'P8', 'FC1']
Done:  s02


Subject Name:  s02
Class Label:  valence
Accuracy in Growing Phase:  0.75
No of selected channels in Growing Phase:  16
Channels selected in Growing Phase:  ['Fp2', 'FC1', 'Cz', 'CP2', 'Fp1', 'P4', 'C4', 'F3', 'O2', 'P3', 'F7', 'FC5', 'O1', 'AF4', 'T7', 'F8']
Class Label:  arousal
Accuracy in Growing Phase:  0.675
No of selected channels in Growing Phase:  8
Channels selected in Growing Phase:  ['F3', 'FC2', 'P7', 'F8', 'Fp1', 'T8', 'CP5', 'P3']
Class Label:  all
Accuracy in 

Done:  s12


Subject Name:  s12
Class Label:  valence
Accuracy in Growing Phase:  0.7
No of selected channels in Growing Phase:  9
Channels selected in Growing Phase:  ['CP1', 'C3', 'CP2', 'P7', 'F3', 'F4', 'Pz', 'Fz', 'F7']
Class Label:  arousal
Accuracy in Growing Phase:  0.85
No of selected channels in Growing Phase:  12
Channels selected in Growing Phase:  ['AF4', 'FC6', 'Fp2', 'Cz', 'PO4', 'F3', 'O1', 'AF3', 'FC1', 'C4', 'PO3', 'O2']
Class Label:  all
Accuracy in Growing Phase:  0.625
No of selected channels in Growing Phase:  4
Channels selected in Growing Phase:  ['CP1', 'FC2', 'CP5', 'Oz']
Done:  s13


Subject Name:  s13
Class Label:  valence
Accuracy in Growing Phase:  0.7
No of selected channels in Growing Phase:  11
Channels selected in Growing Phase:  ['P7', 'P4', 'AF3', 'Cz', 'Oz', 'FC2', 'Pz', 'F7', 'P8', 'F3', 'F4']
Class Label:  arousal
Accuracy in Growing Phase:  0.85
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['PO3', 'P3', 'T8', 

Done:  s22


Subject Name:  s22
Class Label:  valence
Accuracy in Growing Phase:  0.725
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['O2', 'C4', 'F3', 'P4', 'Fz', 'AF4']
Class Label:  arousal
Accuracy in Growing Phase:  0.675
No of selected channels in Growing Phase:  7
Channels selected in Growing Phase:  ['Oz', 'FC2', 'FC5', 'CP2', 'CP6', 'P7', 'F7']
Class Label:  all
Accuracy in Growing Phase:  0.5
No of selected channels in Growing Phase:  17
Channels selected in Growing Phase:  ['FC5', 'C4', 'Fz', 'O2', 'P4', 'AF3', 'FC1', 'CP5', 'FC2', 'Oz', 'AF4', 'C3', 'Fp1', 'F4', 'CP2', 'O1', 'F7']
Done:  s23


Subject Name:  s23
Class Label:  valence
Accuracy in Growing Phase:  0.7
No of selected channels in Growing Phase:  6
Channels selected in Growing Phase:  ['PO4', 'Oz', 'Fz', 'CP2', 'PO3', 'FC1']
Class Label:  arousal
Accuracy in Growing Phase:  0.775
No of selected channels in Growing Phase:  10
Channels selected in Growing Phase:  ['T7', 'O1', 'O

KeyboardInterrupt: 